<a href="https://colab.research.google.com/github/docheem/NLP-Portfolio/blob/main/PR_Code_Gen_Chatbot_Dialog_w_Gradio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Zero-Shot Text Classification, Code Generation and Chatbot_Dialog_Bot x Gradio

In [ ]:
!pip install transformers
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from transformers import BartForSequenceClassification,BartTokenizer
import gradio as grad


bart_tkn = BartTokenizer.from_pretrained('facebook/bart-large-mnli')

mdl = BartForSequenceClassification.from_pretrained('facebook/bart-large-mnli')



def classify(text,label):

    tkn_ids = bart_tkn.encode(text,

                              label,

                              return_tensors='pt')


    tkn_lgts = mdl(tkn_ids)[0]



    entail_contra_tkn_lgts = tkn_lgts[:,[0,2]]

    probab = entail_contra_tkn_lgts.softmax(dim = 1)

    response =  probab[:,1].item() * 100

    return response



txt=grad.Textbox(lines = 1,

                 label = "English",

                 placeholder = "text tobe classified")


labels = grad.Textbox(lines = 1,

                      label="Label",

                      placeholder="Inputa Label")

out = grad.Textbox(lines = 1,

                   label = "Probablity of label being true is")


grad.Interface(classify, inputs = [txt,labels],
               outputs = out).launch()

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

# Code Generator

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import gradio as grad

codegen_tkn = AutoTokenizer.from_pretrained("Salesforce/codegen-350M-mono")

mdl = AutoModelForCausalLM.from_pretrained("Salesforce/codegen-350M-mono")



def codegen(intent):
    # give input as text which reflects intent of the program.
    #text = " write a function which takes 2 numbers as input and returns the larger of the two"
    input_ids = codegen_tkn(intent,
                            return_tensors="pt").input_ids

    gen_ids = mdl.generate(input_ids,
                           max_length=256)

    response = codegen_tkn.decode(gen_ids[0],
                                  skip_special_tokens=True)

    return response

output = grad.Textbox(lines=1,
                      label = "Generated Python Code",
                      placeholder="")

inp = grad.Textbox(lines=1,
                   label = "Place your intent here")

grad.Interface(codegen,
               inputs=inp,
               outputs=output).launch()

text = """def merge_sort(unsorted:list):
"""
input_ids = codegen_tkn(text,
                        return_tensors = "pt").input_ids

gen_ids = mdl.generate(input_ids,
                       max_length=256)

print(codegen_tkn.decode(gen_ids[0],
                         skip_special_tokens = True))





Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
/usr/local/lib/python3.9/dist-packages/transformers/models/codegen/modeling_codegen.py:166: UserWarning: where received a uint8 condition tensor. This behavior is deprecated and will be removed in a future version of PyTorch. Use a boolean condition instead. (Triggered internally at ../aten/src/ATen/native/TensorCompare.cpp:413.)
  attn_weights = torch.where(causal_mask, attn_weights, mask_value)
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


def merge_sort(unsorted:list):
    if len(unsorted) <= 1:
        return unsorted
    mid = len(unsorted) // 2
    left = merge_sort(unsorted[:mid])
    right = merge_sort(unsorted[mid:])
    return merge(left, right)

def merge(left, right):
    result = []
    while left and right:
        if left[0] < right[0]:
            result.append(left[0])
            left.pop(0)
        else:
            result.append(right[0])
            right.pop(0)
    if left:
        result += left
    if right:
        result += right
    return result

def quick_sort(unsorted:list):
    if len(unsorted) <= 1:
        return unsorted
    pivot = unsorted[0]
    left = [i for i in unsorted if i < pivot]
    right = [i for i in unsorted if i > pivot]
    return quick_sort(left) + [pivot] +


#Chatbot_Dialog_Bot

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer,BlenderbotForConditionalGeneration
import torch



#chat_tkn = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")

#mdl = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")


def converse(user_input, chat_history=[]):

    user_input_ids = chat_tkn(user_input +
                              chat_tkn.eos_token,
                              return_tensors='pt').input_ids

    # keep history in the tensor
    bot_input_ids = torch.cat([torch.LongTensor(chat_history),
                               user_input_ids],
                              dim=-1)

    # get response
    chat_history = mdl.generate(bot_input_ids,
                                max_length=1000,
                                pad_token_id = chat_tkn.eos_token_id).tolist()
    print (chat_history)

    response = chat_tkn.decode(chat_history[0]).split("<|endoftext|>")

    print("starting to print response")
    print(response)

    # html for display
    html = "<div class='mybot'>"

    for x, mesg in enumerate(response):

        if x%2!=0 :
           mesg = "Alicia:"+mesg
           clazz = "alicia"
        else :
           clazz = "user"

        print("value of x")
        print(x)
        print("message")

        print (mesg)

        html += "<div class='mesg {}'> {}</div>".format(clazz, mesg)
        html += "</div>"
        print(html)
        return html, chat_history


In [ ]:
import gradio as grad

css = """
.mychat {display:flex;flex-direction:column}
.mesg {padding:5px;margin-bottom:5px;border-radius:5px;width:75%}
.mesg.user {background-color:lightblue;color:white}
.mesg.alicia {background-color:orange;color:white,align-self:self-end}
.footer {display:none !important}
"""
text = grad.inputs.Textbox(placeholder="Lets chat")

grad.Interface(fn = converse,
               theme = "default",
               inputs = [text, "state"],
               outputs = ["html", "state"],
               css = css).launch()


/usr/local/lib/python3.9/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.9/dist-packages/gradio/blocks.py:491: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>